# Evaluation Plots

This notebook loads artifacts from `artifacts/` and renders plots used in the project evaluation.


In [ ]:
from pathlib import Path
import json
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path(r"c:/Users/zayke/fake_news_ml_ai_project")
ART = ROOT / "artifacts"
plt.style.use("seaborn-v0_8")


## Baseline metrics


In [ ]:
baseline_path = ART / "baseline" / "baseline_results.json"
if baseline_path.exists():
    baseline = json.loads(baseline_path.read_text(encoding="utf-8"))
    rows = []
    for name, res in baseline.get("models", {}).items():
        m = res.get("metrics", {})
        rows.append({"model": name, **m})
    df = pd.DataFrame(rows)
    df = df.set_index("model")
    ax = df[["accuracy", "precision", "recall", "f1", "roc_auc"]].plot(kind="bar", figsize=(10, 4))
    ax.set_title("Baseline metrics")
    ax.set_ylabel("Score")
    ax.legend(loc="lower right")
    plt.tight_layout()
else:
    print("Missing baseline_results.json")


## Calibration curves (saved images)


In [ ]:
calib_dir = ART / "calibration"
pngs = sorted(calib_dir.glob("*.png"))
if pngs:
    for p in pngs:
        img = plt.imread(p)
        plt.figure(figsize=(4, 4))
        plt.imshow(img)
        plt.axis("off")
        plt.title(p.name)
    plt.show()
else:
    print("No calibration plots found in artifacts/calibration")


## Robustness results


In [ ]:
rob_path = ART / "robustness" / "robustness_results.csv"
if rob_path.exists():
    rob = pd.read_csv(rob_path)
    rob = rob.sort_values("f1", ascending=False)
    ax = rob.plot(x="variant", y=["accuracy", "f1"], kind="bar", figsize=(10, 4))
    ax.set_title("Robustness: Accuracy and F1 by variant")
    ax.set_ylabel("Score")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
else:
    print("Missing robustness_results.csv")


## TF-IDF ablations


In [ ]:
ab_path = ART / "tfidf_ablation" / "tfidf_ablation_results.csv"
if ab_path.exists():
    ab = pd.read_csv(ab_path)
    fig, ax = plt.subplots(figsize=(10, 4))
    for model_name, grp in ab.groupby("model"):
        grp = grp.sort_values("f1", ascending=False)
        ax.plot(grp["ablation"], grp["f1"], marker="o", label=model_name)
    ax.set_title("TF-IDF ablations: F1 by view")
    ax.set_ylabel("F1")
    ax.set_xlabel("Ablation")
    plt.xticks(rotation=45, ha="right")
    ax.legend()
    plt.tight_layout()
else:
    print("Missing tfidf_ablation_results.csv")


## Cross-dataset generalization


In [ ]:
cross_path = ART / "cross_dataset" / "cross_dataset_results.json"
if cross_path.exists():
    cross = json.loads(cross_path.read_text(encoding="utf-8"))
    rows = []
    for direction, entry in cross.get("directions", {}).items():
        for model_name, res in entry.get("models", {}).items():
            m = res.get("metrics", {})
            rows.append({"direction": direction, "model": model_name, **m})
    df = pd.DataFrame(rows)
    if not df.empty:
        pivot = df.pivot(index="direction", columns="model", values="f1")
        ax = pivot.plot(kind="bar", figsize=(8, 4))
        ax.set_title("Cross-dataset generalization (F1)")
        ax.set_ylabel("F1")
        plt.xticks(rotation=0)
        plt.tight_layout()
    else:
        print("No cross-dataset metrics found")
else:
    print("Missing cross_dataset_results.json")
